In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import tensorflow as tf
import tensorflow_probability as tfp
import h5py
import random
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
import photoz_utils

In [3]:
tfpl = tfp.layers
tf1 = tf.compat.v1
tfd = tfp.distributions

In [4]:
from tensorflow_probability.python.distributions import kl_divergence

In [5]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  # Restrict TensorFlow to only allocate 10GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [6]:
n = 10000
t = 2000
v = 2000

In [7]:
hf_train = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_training_small.hdf5', 'r')
hf_test = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_testing_small.hdf5', 'r')
hf_validation = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_validation_small.hdf5', 'r')
x_train = np.asarray(np.transpose(hf_train['image'][0:n],(0,2,3,1)))
x_test = np.asarray(np.transpose(hf_test['image'][0:t],(0,2,3,1)))
x_validation = np.asarray(np.transpose(hf_validation['image'][0:v],(0,2,3,1)))
max_value = np.max(np.array([np.max(x_train), np.max(x_test), np.max(x_validation)]))
x_train = np.true_divide(x_train,max_value)
x_test = np.true_divide(x_test,max_value)
x_validation = np.true_divide(x_validation,max_value)
y_train = np.asarray(hf_train['specz_redshift'][0:n])[..., None]
y_test = np.asarray(hf_test['specz_redshift'][0:t])[..., None]
y_validation = np.asarray(hf_validation['specz_redshift'][0:v])[..., None]
object_id_train = np.asarray(hf_train['object_id'][0:n])
object_id = np.asarray(hf_test['object_id'][0:t])
object_id_validation = np.asarray(hf_validation['object_id'][0:v])
hf_train.close()
hf_test.close()
hf_validation.close()

In [8]:
def posterior_mean_field(kernel_size: int, bias_size: int, dtype: any) -> tf.keras.Model:
    """Specify the surrogate posterior over `keras.layers.Dense` `kernel` and `bias`."""
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.))

    return tf.keras.Sequential([
        tfp.layers.VariableLayer(2 * n, dtype=dtype, initializer=lambda shape, dtype: random_gaussian_initializer(shape, dtype), trainable=True),
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t[..., :n],
                       scale= 0 + tf.nn.softplus(c + t[..., n:])),
            reinterpreted_batch_ndims=1)),
    ])

def prior_trainable(kernel_size: int, bias_size: int, dtype: any) -> tf.keras.Model:
    """Specify the prior over `keras.layers.Dense` `kernel` and `bias`."""
    n = kernel_size + bias_size

    return tf.keras.Sequential([
        tfp.layers.VariableLayer(n, dtype=dtype),  # Returns a trainable variable of shape n, regardless of input
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t, scale=0.5),
            reinterpreted_batch_ndims=1)),
    ])

def random_gaussian_initializer(shape, dtype):
    n = int(shape / 2)
    loc_norm = tf.random_normal_initializer(mean=0., stddev=0.1)
    loc = tf.Variable(
        initial_value=loc_norm(shape=(n,), dtype=dtype)
    )
    scale_norm = tf.random_normal_initializer(mean=-3., stddev=0.1)
    scale = tf.Variable(
        initial_value=scale_norm(shape=(n,), dtype=dtype)
    )
    return tf.concat([loc, scale], 0)

kl_divergence_function = lambda q, p, _: kl_divergence(q, p) / n

In [9]:
import keras
from keras.models import Model
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, InputLayer, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

In [10]:
model_variational = Sequential([
    InputLayer(input_shape = (127, 127, 5)),
    tfpl.Convolution2DFlipout(32, kernel_size=(2, 2), activation = 'tanh', kernel_divergence_fn = kl_divergence_function),
    MaxPooling2D(2, 2),
    Conv2D(32, kernel_size = (2, 2), activation = 'tanh'),
    MaxPooling2D(2, 2),
    Conv2D(32, kernel_size = (2, 2), activation = 'relu'),
    Conv2D(32, kernel_size = (2, 2), activation = 'relu'),
    Flatten(),
    Dropout(0.2),
    tfpl.DenseVariational(64, 
                                      make_posterior_fn = posterior_mean_field,
                                      make_prior_fn = prior_trainable,
                                      kl_weight = 1 / n),
    tfpl.DenseVariational(64, 
                                      make_posterior_fn = posterior_mean_field,
                                      make_prior_fn = prior_trainable,
                                      kl_weight = 1 / n),
    Dense(tfpl.IndependentNormal.params_size(1)),
    tfpl.IndependentNormal(1)
])

model_variational.build((10000, 127, 127, 5))

model_variational.summary()

/opt/anaconda3/envs/jupyterhub/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:2191: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (10000, 127, 127, 5)      0         
_________________________________________________________________
conv2d_flipout (Conv2DFlipou (10000, 126, 126, 32)     1312      
_________________________________________________________________
module_wrapper_1 (ModuleWrap (10000, 63, 63, 32)       0         
_________________________________________________________________
module_wrapper_2 (ModuleWrap (10000, 62, 62, 32)       4128      
_________________________________________________________________
module_wrapper_3 (ModuleWrap (10000, 31, 31, 32)       0         
_________________________________________________________________
module_wrapper_4 (ModuleWrap (10000, 30, 30, 32)       4128      
_________________________________________________________________
module_wrapper_5 (ModuleWrap (10000, 29, 29, 32)       4

In [11]:
def loss_function(targets, estimated_distribution):
    return - estimated_distribution.log_prob(targets) / (1 + targets)**2

model_variational.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = loss_function, metrics = [keras.metrics.RootMeanSquaredError()])

In [12]:
import tensorboard

model_name = 'HSC_v6_BCNN_v1'

checkpoint_filepath = os.path.join('/models/', model_name) + '/' + model_name

log_dir = os.path.join('/logs/', model_name)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_filepath, save_weights_only = True, verbose = 1, save_freq = 150*250)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [ ]:
model_variational.fit(x = x_train, y = y_train, epochs = 1200, shuffle = True, verbose = 1, validation_data = (x_validation, y_validation), callbacks = [tensorboard_callback, cp_callback])

Epoch 1/1200
313/313 [==============================] - 420s 16ms/step - loss: 215.2899 - root_mean_squared_error: 4.5544 - val_loss: 199.2974 - val_root_mean_squared_error: 3.5013
Epoch 2/1200
313/313 [==============================] - 4s 13ms/step - loss: 184.3279 - root_mean_squared_error: 2.8961 - val_loss: 169.2293 - val_root_mean_squared_error: 2.5089
Epoch 3/1200
313/313 [==============================] - 4s 13ms/step - loss: 154.6465 - root_mean_squared_error: 2.2803 - val_loss: 140.2384 - val_root_mean_squared_error: 2.1025
Epoch 4/1200
313/313 [==============================] - 4s 13ms/step - loss: 126.8491 - root_mean_squared_error: 1.9765 - val_loss: 113.9050 - val_root_mean_squared_error: 1.8712
Epoch 5/1200
313/313 [==============================] - 4s 13ms/step - loss: 102.3988 - root_mean_squared_error: 1.7900 - val_loss: 91.5362 - val_root_mean_squared_error: 1.7182
Epoch 6/1200
313/313 [==============================] - 4s 13ms/step - loss: 82.3485 - root_mean_squared

313/313 [==============================] - 4s 11ms/step - loss: 0.6980 - root_mean_squared_error: 0.9824 - val_loss: 0.6878 - val_root_mean_squared_error: 0.9809
Epoch 47/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6930 - root_mean_squared_error: 0.9792 - val_loss: 0.6898 - val_root_mean_squared_error: 0.9779
Epoch 48/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6894 - root_mean_squared_error: 0.9766 - val_loss: 0.6819 - val_root_mean_squared_error: 0.9752
Epoch 49/1200
313/313 [==============================] - 4s 12ms/step - loss: 0.6842 - root_mean_squared_error: 0.9739 - val_loss: 0.6787 - val_root_mean_squared_error: 0.9727
Epoch 50/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6804 - root_mean_squared_error: 0.9716 - val_loss: 0.6716 - val_root_mean_squared_error: 0.9703
Epoch 51/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6783 - root_mean_squared_error: 0.9691 - val_loss: 0.6

313/313 [==============================] - 4s 11ms/step - loss: 0.6786 - root_mean_squared_error: 0.9194 - val_loss: 0.6769 - val_root_mean_squared_error: 0.9190
Epoch 93/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6856 - root_mean_squared_error: 0.9187 - val_loss: 0.6802 - val_root_mean_squared_error: 0.9183
Epoch 94/1200
313/313 [==============================] - 4s 12ms/step - loss: 0.6801 - root_mean_squared_error: 0.9180 - val_loss: 0.6786 - val_root_mean_squared_error: 0.9177
Epoch 95/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6815 - root_mean_squared_error: 0.9174 - val_loss: 0.6764 - val_root_mean_squared_error: 0.9170
Epoch 96/1200
313/313 [==============================] - 4s 12ms/step - loss: 0.6804 - root_mean_squared_error: 0.9167 - val_loss: 0.6696 - val_root_mean_squared_error: 0.9165
Epoch 97/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6791 - root_mean_squared_error: 0.9163 - val_loss: 0.6

313/313 [==============================] - 4s 12ms/step - loss: 0.6655 - root_mean_squared_error: 0.8960 - val_loss: 0.6644 - val_root_mean_squared_error: 0.8958
Epoch 138/1200
313/313 [==============================] - 4s 12ms/step - loss: 0.6659 - root_mean_squared_error: 0.8956 - val_loss: 0.6575 - val_root_mean_squared_error: 0.8954
Epoch 139/1200
313/313 [==============================] - 4s 12ms/step - loss: 0.6655 - root_mean_squared_error: 0.8952 - val_loss: 0.6597 - val_root_mean_squared_error: 0.8950
Epoch 140/1200
313/313 [==============================] - 4s 12ms/step - loss: 0.6656 - root_mean_squared_error: 0.8948 - val_loss: 0.6547 - val_root_mean_squared_error: 0.8946
Epoch 141/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6678 - root_mean_squared_error: 0.8944 - val_loss: 0.6626 - val_root_mean_squared_error: 0.8942
Epoch 142/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6681 - root_mean_squared_error: 0.8940 - val_loss

313/313 [==============================] - 4s 12ms/step - loss: 0.6666 - root_mean_squared_error: 0.8815 - val_loss: 0.6630 - val_root_mean_squared_error: 0.8814
Epoch 184/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6663 - root_mean_squared_error: 0.8813 - val_loss: 0.6654 - val_root_mean_squared_error: 0.8812
Epoch 185/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6685 - root_mean_squared_error: 0.8811 - val_loss: 0.6603 - val_root_mean_squared_error: 0.8810
Epoch 186/1200
313/313 [==============================] - 4s 12ms/step - loss: 0.6631 - root_mean_squared_error: 0.8809 - val_loss: 0.6689 - val_root_mean_squared_error: 0.8807
Epoch 187/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6674 - root_mean_squared_error: 0.8806 - val_loss: 0.6637 - val_root_mean_squared_error: 0.8805
Epoch 188/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6674 - root_mean_squared_error: 0.8804 - val_loss

313/313 [==============================] - 4s 12ms/step - loss: 0.6861 - root_mean_squared_error: 0.8761 - val_loss: 0.6933 - val_root_mean_squared_error: 0.8760
Epoch 230/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6935 - root_mean_squared_error: 0.8760 - val_loss: 0.6885 - val_root_mean_squared_error: 0.8760
Epoch 231/1200
313/313 [==============================] - 4s 12ms/step - loss: 0.6849 - root_mean_squared_error: 0.8759 - val_loss: 0.6823 - val_root_mean_squared_error: 0.8759
Epoch 232/1200
313/313 [==============================] - 4s 12ms/step - loss: 0.6902 - root_mean_squared_error: 0.8759 - val_loss: 0.6691 - val_root_mean_squared_error: 0.8759
Epoch 233/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6893 - root_mean_squared_error: 0.8759 - val_loss: 0.6807 - val_root_mean_squared_error: 0.8758
Epoch 234/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6834 - root_mean_squared_error: 0.8758 - val_loss

313/313 [==============================] - 4s 11ms/step - loss: 0.6598 - root_mean_squared_error: 0.8731 - val_loss: 0.6550 - val_root_mean_squared_error: 0.8731
Epoch 275/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6596 - root_mean_squared_error: 0.8730 - val_loss: 0.6526 - val_root_mean_squared_error: 0.8729
Epoch 276/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6604 - root_mean_squared_error: 0.8729 - val_loss: 0.6578 - val_root_mean_squared_error: 0.8728
Epoch 277/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6606 - root_mean_squared_error: 0.8728 - val_loss: 0.6562 - val_root_mean_squared_error: 0.8727
Epoch 278/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6609 - root_mean_squared_error: 0.8726 - val_loss: 0.6564 - val_root_mean_squared_error: 0.8725
Epoch 279/1200
313/313 [==============================] - 4s 12ms/step - loss: 0.6607 - root_mean_squared_error: 0.8725 - val_loss

313/313 [==============================] - 4s 11ms/step - loss: 0.6654 - root_mean_squared_error: 0.8682 - val_loss: 0.6635 - val_root_mean_squared_error: 0.8681
Epoch 321/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6649 - root_mean_squared_error: 0.8681 - val_loss: 0.6581 - val_root_mean_squared_error: 0.8681
Epoch 322/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6667 - root_mean_squared_error: 0.8680 - val_loss: 0.6612 - val_root_mean_squared_error: 0.8680
Epoch 323/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6664 - root_mean_squared_error: 0.8679 - val_loss: 0.6642 - val_root_mean_squared_error: 0.8679
Epoch 324/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6669 - root_mean_squared_error: 0.8678 - val_loss: 0.6671 - val_root_mean_squared_error: 0.8678
Epoch 325/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6660 - root_mean_squared_error: 0.8677 - val_loss

313/313 [==============================] - 4s 11ms/step - loss: 0.6656 - root_mean_squared_error: 0.8647 - val_loss: 0.6685 - val_root_mean_squared_error: 0.8646
Epoch 366/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6634 - root_mean_squared_error: 0.8646 - val_loss: 0.6569 - val_root_mean_squared_error: 0.8646
Epoch 367/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6678 - root_mean_squared_error: 0.8645 - val_loss: 0.6615 - val_root_mean_squared_error: 0.8645
Epoch 368/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6682 - root_mean_squared_error: 0.8645 - val_loss: 0.6598 - val_root_mean_squared_error: 0.8644
Epoch 369/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6654 - root_mean_squared_error: 0.8644 - val_loss: 0.6620 - val_root_mean_squared_error: 0.8644
Epoch 370/1200
313/313 [==============================] - 4s 11ms/step - loss: 0.6656 - root_mean_squared_error: 0.8643 - val_loss

In [ ]:
model_variational.save(checkpoint_filepath, save_traces = False)

model_variational.load_weights(checkpoint_filepath)

In [ ]:
photoz = []

for i in range(t):
    photoz.append(model_variational(np.array([x_test[i]])).mean()[0][0])

photoz = np.asarray(photoz).astype('float32')

In [ ]:
photoz_train = []

for i in range(n):
    photoz_train.append(model_variational(np.array([x_train[i]])).mean()[0][0])

photoz_train = np.asarray(photoz_train).astype('float32')

In [ ]:
photoz_validation = []

for i in range(v):
    photoz_validation.append(model_variational(np.array([x_validation[i]])).mean()[0][0])

photoz_validation = np.asarray(photoz_validation).astype('float32')

In [ ]:
specz = np.asarray(y_test)[:,0]
specz_train = np.asarray(y_train)[:,0]
specz_validation = np.asarray(y_validation)[:,0]

In [ ]:
photoz_err = []

for i in range(t):
    photoz_err.append(model_variational(np.array([x_test[i]])).stddev()[0][0])

In [ ]:
photoz_err_train = []

for i in range(n):
    photoz_err_train.append(model_variational(np.array([x_train[i]])).stddev()[0][0])

In [ ]:
photoz_err_validation = []

for i in range(v):
    photoz_err_validation.append(model_variational(np.array([x_validation[i]])).stddev()[0][0])

In [ ]:
from scipy.stats import gaussian_kde
xy = np.asarray(np.vstack([specz, photoz])).astype('float32')
z = gaussian_kde(xy)(xy)

In [ ]:
import seaborn as sns
z_max = 4
sns.set(rc={'figure.figsize':(10, 10), 'lines.markersize':20})
plt.rcParams.update({'font.size': 40})
fig, ax = plt.subplots()
scatter_plot = ax.scatter(specz, photoz, c = z, s = 1, edgecolor = None)
plt.colorbar(scatter_plot, label = 'Density')
plt.title('Prediction')
plt.xlabel('spectroscopic redshift')
plt.ylabel('photo z')
plt.plot([0, z_max], [0, z_max],color = 'black')
plt.xlim([0, z_max])
plt.ylim([0, z_max])
plt.show()

In [ ]:
overlap_variational = 0
overlap_array_variational = []
for i in range(0,t):
    if np.abs(specz[i]-photoz[i])<=photoz_err[i]:
        overlap_variational += 1
        overlap_array_variational.append(1)
    else:
        overlap_array_variational.append(0)

In [ ]:
fig, ax = plt.subplots()
display_size = 100
scatter_plot = ax.scatter(specz[:display_size], photoz[:display_size], c = z[:display_size], s = 4, edgecolor = None, zorder = 2)
error_plot = ax.errorbar(specz[:display_size], photoz[:display_size], yerr = photoz_err[:display_size], fmt = "o", markersize = 0, color = 'blue', elinewidth = 1, zorder = 1)
plt.title('Prediction with error, Bayesian')
plt.xlabel('spectroscopic redshift')
plt.ylabel('photo z')
plt.plot([0, z_max], [0, z_max], color = 'black')
plt.xlim([0, z_max])
plt.ylim([0, z_max])
print("Coverage: "+str(overlap_variational/t))

In [ ]:
combined_array_variational = np.transpose(np.vstack((specz, photoz, photoz_err, overlap_array_variational))) 

In [ ]:
sorted_combined_array_variational = sorted(combined_array_variational, key=lambda x: x[0])

In [ ]:
bins_variational = int(z_max*5)
splitted_sorted_combined_array_variational = np.array_split(sorted_combined_array_variational, bins_variational)
coverage_variational = []
for i in range(0, bins_variational):
    bins_count_variational = 0
    for j in range(0,len(splitted_sorted_combined_array_variational[i])):
        if splitted_sorted_combined_array_variational[i][j][3] == 1:
            bins_count_variational += 1
    coverage_variational.append(bins_count_variational/len(splitted_sorted_combined_array_variational[i])/0.6827)
x_array = np.arange(0, z_max, z_max/bins_variational)

In [ ]:
plt.plot(x_array, coverage_variational, c = 'red')
plt.title('Coverage, Bayesian')
plt.xlabel('spectroscopic redshift')
plt.ylabel('coverage')
plt.ylim([0, 1.5])
print("Coverage: "+str(overlap_variational/t))

In [ ]:
mean_array = []
for i in range(0, bins_variational):
    total = 0
    for j in range(0, len(splitted_sorted_combined_array_variational[i])):
        bias = (splitted_sorted_combined_array_variational[i][j][1]-splitted_sorted_combined_array_variational[i][j][0])
        total += bias
    mean_array.append(total/len(splitted_sorted_combined_array_variational[i]))
x_array = np.arange(0, z_max, z_max/bins_variational)

In [ ]:
error_array = []
for i in range(0,bins_variational):
    error_total = 0
    for j in range(0,len(splitted_sorted_combined_array_variational[i])):
        error_total += splitted_sorted_combined_array_variational[i][j][2]
    error_array.append(error_total/len(splitted_sorted_combined_array_variational[i]))

In [ ]:
sns.set(rc={'figure.figsize':(5, 5), 'lines.markersize':40})
plt.rcParams.update({'font.size': 40})
plt.errorbar(x_array, mean_array, yerr = error_array, fmt = "o", color = 'blue', markersize = 4, elinewidth = 2)
plt.title('Bias Plot for Probabilistic Model')
plt.xlabel('True Redshift')
plt.ylabel('Bias (Predicted - True)')
plt.plot([0, z_max], [0, 0],color = 'black')
plt.ylim([-1, 1])
plt.show()

In [ ]:
combined_outlier_array = np.transpose(np.vstack((specz,np.subtract(photoz, specz))))
sorted_combined_outlier_array = sorted(combined_outlier_array, key=lambda x: x[0])
bins_outlier = int(z_max*10)
splitted_sorted_combined_outlier_array = np.array_split(sorted_combined_outlier_array,bins_outlier)
outlier_array = []
outlier_count_total = 0
for i in range(0,bins_outlier):
    outlier_count = 0
    for j in range(0,len(splitted_sorted_combined_outlier_array[i])):
        if np.abs(splitted_sorted_combined_outlier_array[i][j][1]/(splitted_sorted_combined_outlier_array[i][j][0] + 1)) >= .15:
            outlier_count += 1
            outlier_count_total += 1
    outlier_array.append(outlier_count/len(specz)*bins_outlier)
x_array_outlier = np.arange(0,z_max,z_max/bins_outlier)
plt.plot(x_array_outlier, outlier_array, c = 'blue')
plt.title('Outlier Rate')
plt.xlabel('spec z')
plt.ylabel('outlier rate')
plt.xlim([0,z_max])
plt.ylim([0,1/2])
plt.show()

print("Outlier Rate: "+str(outlier_count_total/len(specz)))

In [ ]:
photoz_utils.save_with_oid_std(model_name, object_id, specz, photoz, photoz_err)
photoz_utils.save_train_with_oid_std(model_name, object_id_train, specz_train, photoz_train, photoz_err_train)
photoz_utils.save_validation_with_oid_std(model_name, object_id_validation, specz_validation, photoz_validation, photoz_err_validation)

In [ ]:
# END